In [1]:
import re
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.base import clone

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# --- Load data ---
con_df = pd.read_csv(r"E:\USA_PD_2024\Analysis\ppr6\COP\ML\Feature_Selection_2\Data\Controlled1.csv")
pd_df = pd.read_csv(r"E:\USA_PD_2024\Analysis\ppr6\COP\ML\Feature_Selection_2\Data\PD1.csv")

con_df["label"] = 0
pd_df["label"] = 1
df = pd.concat([con_df, pd_df], ignore_index=True)

# --- Extract SubjectID from File name ---
def get_id(fname):
    base = Path(fname).stem
    m = re.search(r"_[a-zA-Z]*([0-9]+)", base)
    return m.group(1) if m else base

df["SubjectID"] = df["File"].apply(get_id)

# --- Drop non-feature columns: label, File, SubjectID ---
X = df.drop(columns=["label", "File", "SubjectID"])
y = df["label"].values
groups = df["SubjectID"].values

# --- Cross-validation setup ---
cv = GroupKFold(n_splits=5)

# --- Forward Selection based on F1-score ---
def groupwise_sfs(pipe, X, y, groups):
    best_score = 0
    selected = []
    remaining = list(X.columns)
    while remaining:
        best_feat = None
        for feat in remaining:
            trial_feats = selected + [feat]
            fold_scores = []
            for tr, te in cv.split(X[trial_feats], y, groups):
                model = clone(pipe)
                model.fit(X[trial_feats].iloc[tr], y[tr])
                preds = model.predict(X[trial_feats].iloc[te])
                fold_scores.append(f1_score(y[te], preds))
            mean_score = np.mean(fold_scores)
            if mean_score > best_score:
                best_score = mean_score
                best_feat = feat
        if best_feat:
            selected.append(best_feat)
            remaining.remove(best_feat)
        else:
            break
    return selected

# --- Define models and hyperparameters ---
models = [
    ("SVM (RBF)", Pipeline([("scaler", StandardScaler()), ("clf", SVC(kernel="rbf", probability=True, random_state=42))]),
     {"clf__C": [0.1, 1, 10], "clf__gamma": ["scale", 0.1]}),

    ("Random Forest", Pipeline([("clf", RandomForestClassifier(random_state=42, n_jobs=-1))]),
     {"clf__n_estimators": [100], "clf__max_depth": [None]}),

    ("Logistic Regression", Pipeline([("scaler", StandardScaler()), ("clf", LogisticRegression(max_iter=500, random_state=42))]),
     {"clf__C": [1]}),

    ("k-NN", Pipeline([("scaler", StandardScaler()), ("clf", KNeighborsClassifier())]),
     {"clf__n_neighbors": [5, 7]}),

    ("Gaussian NB", Pipeline([("clf", GaussianNB())]),
     {})
]

# --- Train and Evaluate each model ---
for name, pipe, param_grid in models:
    print(f"\n--- {name} Feature Selection ---")
    selected_feats = groupwise_sfs(pipe, X, y, groups)
    print(f"Selected features: {selected_feats}")

    X_sel = X[selected_feats]

    # Hyperparameter tuning
    gs = GridSearchCV(pipe, param_grid, scoring="f1", cv=cv, n_jobs=-1)
    gs.fit(X_sel, y, groups=groups)
    best_est = gs.best_estimator_

    # Evaluation with best estimator
    accs, precs, recs, f1s, aucs = [], [], [], [], []
    for tr, te in cv.split(X_sel, y, groups):
        best_est.fit(X_sel.iloc[tr], y[tr])
        preds = best_est.predict(X_sel.iloc[te])
        probs = (best_est.predict_proba(X_sel.iloc[te])[:, 1]
                 if hasattr(best_est, "predict_proba")
                 else best_est.decision_function(X_sel.iloc[te]))

        accs.append(accuracy_score(y[te], preds))
        precs.append(precision_score(y[te], preds))
        recs.append(recall_score(y[te], preds))
        f1s.append(f1_score(y[te], preds))
        aucs.append(roc_auc_score(y[te], probs))

    print(f"=== {name} ===")
    print(f"Best params: {gs.best_params_}")
    print(f"Accuracy : {np.mean(accs):.3f} ± {np.std(accs):.3f}")
    print(f"Precision: {np.mean(precs):.3f} ± {np.std(precs):.3f}")
    print(f"Recall   : {np.mean(recs):.3f} ± {np.std(recs):.3f}")
    print(f"F1-score : {np.mean(f1s):.3f} ± {np.std(f1s):.3f}")
    print(f"ROC-AUC  : {np.mean(aucs):.3f} ± {np.std(aucs):.3f}")



--- SVM (RBF) Feature Selection ---
Selected features: ['Feature_asym_energy_content_below_05_Power_Spectrum_Density_AP', 'Feature_avg_fractal_dimension_ML_AND_AP', 'Feature_avg_frequency_quotient_Power_Spectrum_Density_ML', 'Feature_avg_short_time_diffusion_Diffusion_ML', 'Feature_asym_power_frequency_50_Power_Spectrum_Density_ML', 'Feature_asym_coefficient_sway_direction_ML_AND_AP', 'Feature_asym_mean_velocity_ML_AND_AP']
=== SVM (RBF) ===
Best params: {'clf__C': 1, 'clf__gamma': 'scale'}
Accuracy : 0.852 ± 0.098
Precision: 0.862 ± 0.171
Recall   : 0.847 ± 0.051
F1-score : 0.845 ± 0.093
ROC-AUC  : 0.856 ± 0.110

--- Random Forest Feature Selection ---
Selected features: ['Feature_asym_energy_content_below_05_Power_Spectrum_Density_AP', 'Feature_asym_mean_value_ML', 'Feature_asym_frequency_quotient_Power_Spectrum_Density_ML', 'Feature_asym_frequency_dispersion_Power_Spectrum_Density_ML', 'Feature_asym_phase_plane_parameter_ML', 'Feature_asym_range_ML']
=== Random Forest ===
Best para

In [5]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV


# Redefine models
models = [
    ("SVM (RBF)", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="rbf", probability=True, random_state=42))
    ]), {"clf__C": [0.1, 1, 10], "clf__gamma": ["scale", 0.1]}),

    ("Random Forest", Pipeline([
        ("clf", RandomForestClassifier(random_state=42, n_jobs=-1))
    ]), {"clf__n_estimators": [100], "clf__max_depth": [None]}),

    ("Logistic Regression", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=500, random_state=42))
    ]), {"clf__C": [1]}),

    ("k-NN", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier())
    ]), {"clf__n_neighbors": [5, 7]}),

    ("Gaussian NB", Pipeline([
        ("clf", GaussianNB())
    ]), {})
]
# --- NEW: Imports for importance & plotting ---
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import os
import math
import unicodedata

# --------- Helper utilities ---------
def slugify(text):
    """Make safe file names from model names."""
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("ascii")
    return re.sub(r"[^A-Za-z0-9]+", "_", text).strip("_").lower()

def coef_importance_from_pipeline(fitted_pipe):
    """Return absolute coefficient importances if available, else None."""
    clf = fitted_pipe.named_steps.get("clf", None)
    if clf is None:
        # Try last step if not named 'clf'
        try:
            clf = list(fitted_pipe.named_steps.values())[-1]
        except Exception:
            return None
    if hasattr(clf, "coef_"):
        coefs = np.abs(clf.coef_)
        # Binary or multi-class: average across classes
        if coefs.ndim == 1:
            coefs = coefs.reshape(1, -1)
        return coefs.mean(axis=0)
    return None

def native_importance_from_pipeline(fitted_pipe):
    """Return tree-based feature_importances_ if available, else None."""
    clf = fitted_pipe.named_steps.get("clf", None)
    if clf is None:
        try:
            clf = list(fitted_pipe.named_steps.values())[-1]
        except Exception:
            return None
    if hasattr(clf, "feature_importances_"):
        return np.asarray(clf.feature_importances_).ravel()
    return None

def plot_importance_bar(feature_names, means, stds, title, outfile, top_k=20):
    """Create and save a horizontal bar chart with error bars (mean ± SD)."""
    # Sort by mean importance descending
    order = np.argsort(means)[::-1]
    feature_names = np.array(feature_names)[order]
    means = np.array(means)[order]
    stds = np.array(stds)[order]

    k = min(top_k, len(feature_names))
    fn = feature_names[:k][::-1]  # reverse for horizontal plot
    mu = means[:k][::-1]
    sd = stds[:k][::-1]

    plt.figure(figsize=(8, max(4, 0.35 * k + 1)))
    y_pos = np.arange(k)
    plt.barh(y_pos, mu, xerr=sd, align='center')
    plt.yticks(y_pos, fn)
    plt.xlabel('Normalized importance (mean ± SD)')
    plt.title(title)
    plt.tight_layout()
    plt.savefig(outfile, dpi=300)
    plt.close()

def compute_fold_importances(fitted_pipe, X_val, y_val, scoring="f1", n_repeats=30, random_state=42):
    """
    Compute permutation importance on held-out fold.
    Returns (importances_mean, importances_std).
    """
    r = permutation_importance(
        fitted_pipe, X_val, y_val,
        scoring=scoring, n_repeats=n_repeats,
        random_state=random_state, n_jobs=-1
    )
    return r.importances_mean, r.importances_std

# --- Directory to save outputs ---
OUT_DIR = "feature_importance_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# --- Parameters you can tweak ---
PERM_SCORING = "f1"       # or "roc_auc"
PERM_REPEATS = 30         # higher => smoother estimates, slower
TOP_K_PLOT = 20           # top-k features to plot

# ================== MAIN: Importance per model ==================
for name, pipe, param_grid in models:
    print(f"\n--- {name}: Feature Selection (SFS) ---")
    selected_feats = groupwise_sfs(pipe, X, y, groups)
    print(f"Selected features ({len(selected_feats)}): {selected_feats}")

    X_sel = X[selected_feats]

    # Hyperparameter tuning with group-aware CV
    gs = GridSearchCV(pipe, param_grid, scoring="f1", cv=cv, n_jobs=-1, refit=True)
    gs.fit(X_sel, y, groups=groups)
    best_est = gs.best_estimator_
    print(f"{name} best params: {gs.best_params_}")

    # --- Collect fold-wise permutation importances on held-out folds ---
    fold_perm_means = []
    for fold_idx, (tr, te) in enumerate(cv.split(X_sel, y, groups)):
        est = clone(best_est)
        est.fit(X_sel.iloc[tr], y[tr])

        imp_mean, _imp_std = compute_fold_importances(
            est, X_sel.iloc[te], y[te],
            scoring=PERM_SCORING, n_repeats=PERM_REPEATS, random_state=42 + fold_idx
        )
        fold_perm_means.append(imp_mean)

    perm_matrix = np.vstack(fold_perm_means)  # shape: (n_folds, n_features)
    perm_mean = perm_matrix.mean(axis=0)
    perm_std = perm_matrix.std(axis=0)

    # Normalize for readability (sum to 1)
    total = perm_mean.sum()
    if total > 0:
        perm_mean_norm = perm_mean / total
        perm_std_norm = perm_std / total
    else:
        perm_mean_norm = perm_mean
        perm_std_norm = perm_std

    # --- Save CSV for permutation importance ---
    out_slug = slugify(name)
    df_perm = pd.DataFrame({
        "feature": selected_feats,
        "importance_mean": perm_mean_norm,
        "importance_sd": perm_std_norm
    }).sort_values("importance_mean", ascending=False)
    csv_path = os.path.join(OUT_DIR, f"perm_importance_{out_slug}.csv")
    df_perm.to_csv(csv_path, index=False)
    print(f"Saved permutation importance table -> {csv_path}")

    # --- Plot permutation importance (Top-K) ---
    png_path = os.path.join(OUT_DIR, f"perm_importance_{out_slug}.png")
    plot_importance_bar(
        df_perm["feature"].tolist(),
        df_perm["importance_mean"].to_numpy(),
        df_perm["importance_sd"].to_numpy(),
        title=f"{name} — Permutation importance (GroupKFold, {PERM_SCORING})",
        outfile=png_path,
        top_k=TOP_K_PLOT
    )
    print(f"Saved permutation importance figure -> {png_path}")

    # --- OPTIONAL: also save native importances if available (coeffs or feature_importances_) ---
    # Fit once on all data (with selected features) ONLY to extract native importances.
    # (Use with caution in text; for figures, prefer the CV-based permutation results above.)
    est_full = clone(best_est).fit(X_sel, y)
    native_imp = native_importance_from_pipeline(est_full)
    coef_imp = coef_importance_from_pipeline(est_full)

    if native_imp is not None:
        # Normalize
        n_total = native_imp.sum()
        n_imp = native_imp / n_total if n_total > 0 else native_imp
        df_nat = pd.DataFrame({"feature": selected_feats, "importance": n_imp}) \
                    .sort_values("importance", ascending=False)
        csv_nat = os.path.join(OUT_DIR, f"native_importance_{out_slug}.csv")
        df_nat.to_csv(csv_nat, index=False)
        png_nat = os.path.join(OUT_DIR, f"native_importance_{out_slug}.png")
        plot_importance_bar(
            df_nat["feature"].tolist(),
            df_nat["importance"].to_numpy(),
            np.zeros_like(df_nat["importance"].to_numpy()),
            title=f"{name} — Native importance (fit on all data)",
            outfile=png_nat,
            top_k=TOP_K_PLOT
        )
        print(f"Saved native importance (tree-based) -> {csv_nat}, {png_nat}")

    if coef_imp is not None:
        # Normalize absolute coefficients
        c_total = coef_imp.sum()
        c_imp = coef_imp / c_total if c_total > 0 else coef_imp
        df_coef = pd.DataFrame({"feature": selected_feats, "importance": c_imp}) \
                    .sort_values("importance", ascending=False)
        csv_coef = os.path.join(OUT_DIR, f"coef_importance_{out_slug}.csv")
        df_coef.to_csv(csv_coef, index=False)
        png_coef = os.path.join(OUT_DIR, f"coef_importance_{out_slug}.png")
        plot_importance_bar(
            df_coef["feature"].tolist(),
            df_coef["importance"].to_numpy(),
            np.zeros_like(df_coef["importance"].to_numpy()),
            title=f"{name} — |Coefficient| importance (fit on all data)",
            outfile=png_coef,
            top_k=TOP_K_PLOT
        )
        print(f"Saved coefficient-based importance -> {csv_coef}, {png_coef}")



--- SVM (RBF): Feature Selection (SFS) ---
Selected features (7): ['Feature_asym_energy_content_below_05_Power_Spectrum_Density_AP', 'Feature_avg_fractal_dimension_ML_AND_AP', 'Feature_avg_frequency_quotient_Power_Spectrum_Density_ML', 'Feature_avg_short_time_diffusion_Diffusion_ML', 'Feature_asym_power_frequency_50_Power_Spectrum_Density_ML', 'Feature_asym_coefficient_sway_direction_ML_AND_AP', 'Feature_asym_mean_velocity_ML_AND_AP']
SVM (RBF) best params: {'clf__C': 1, 'clf__gamma': 'scale'}
Saved permutation importance table -> feature_importance_outputs\perm_importance_svm_rbf.csv
Saved permutation importance figure -> feature_importance_outputs\perm_importance_svm_rbf.png

--- Random Forest: Feature Selection (SFS) ---
Selected features (6): ['Feature_asym_energy_content_below_05_Power_Spectrum_Density_AP', 'Feature_asym_mean_value_ML', 'Feature_asym_frequency_quotient_Power_Spectrum_Density_ML', 'Feature_asym_frequency_dispersion_Power_Spectrum_Density_ML', 'Feature_asym_phase_p

In [7]:
"""
Binary PD vs Control classification using manually selected CoP + demographic features
──────────────────────────────────────────────────────────────────────────────────────
Features: 15 CoP-derived + Height, Weight, Sex (if available)
Modeling: SVM, RF, LR, k-NN, GNB
Validation: 5-fold GroupKFold using SubjectID
"""

# ──────────────────────────────────────────────────────────────
# 1. Imports
# ──────────────────────────────────────────────────────────────
import re
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.base import clone

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# ──────────────────────────────────────────────────────────────
# 2. Load and concatenate data
# ──────────────────────────────────────────────────────────────
con_df = pd.read_csv(r"E:\USA_PD_2024\Analysis\ppr6\COP\ML\Feature_Selection_2\Data\Controlled1.csv")
pd_df  = pd.read_csv(r"E:\USA_PD_2024\Analysis\ppr6\COP\ML\Feature_Selection_2\Data\PD1.csv")

con_df["label"] = 0
pd_df ["label"] = 1
df = pd.concat([con_df, pd_df], ignore_index=True)

# ──────────────────────────────────────────────────────────────
# 3. Clean column names
# ──────────────────────────────────────────────────────────────
df.columns = df.columns.str.strip().str.replace(" ", "_")

# ──────────────────────────────────────────────────────────────
# 4. Extract Subject ID from filename
# ──────────────────────────────────────────────────────────────
def get_id(fname: str) -> str:
    base = Path(fname).stem
    m = re.search(r"_[a-zA-Z]*([0-9]+)", base)
    return m.group(1) if m else base

df["SubjectID"] = df["File"].apply(get_id)

# ──────────────────────────────────────────────────────────────
# 5. Manually selected features
# ──────────────────────────────────────────────────────────────
manual_features = [
    "Feature_asym_energy_content_below_05_Power_Spectrum_Density_AP",
    "Feature_avg_fractal_dimension_ML_AND_AP",
    "Feature_avg_frequency_quotient_Power_Spectrum_Density_ML",
    "Feature_avg_short_time_diffusion_Diffusion_ML",
    "Feature_asym_power_frequency_50_Power_Spectrum_Density_ML",
    "Feature_asym_coefficient_sway_direction_ML_AND_AP",
    "Feature_asym_mean_velocity_ML_AND_AP"
]

# Check availability
available_features = [f for f in manual_features if f in df.columns]
missing = set(manual_features) - set(available_features)
if missing:
    print(f"⚠️ Warning: Missing features skipped: {missing}")
else:
    print("✅ All selected features are present.")

# ──────────────────────────────────────────────────────────────
# 6. Prepare feature matrix X
# ──────────────────────────────────────────────────────────────
if "Sex" in df.columns:
    X = df[available_features + ["Sex"]].copy()
    X = pd.get_dummies(X, columns=["Sex"], drop_first=True)
else:
    X = df[available_features].copy()
    print("⚠️ 'Sex' column not found. Proceeding without it.")

y = df["label"].values
groups = df["SubjectID"].values

# ──────────────────────────────────────────────────────────────
# 7. Cross-validation setup
# ──────────────────────────────────────────────────────────────
cv = GroupKFold(n_splits=5)

# ──────────────────────────────────────────────────────────────
# 8. Models and parameter grids
# ──────────────────────────────────────────────────────────────
models = [
    ("SVM (RBF)", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="rbf", probability=True, random_state=42))
    ]), {"clf__C": [0.1, 1, 10], "clf__gamma": ["scale", 0.1]}),

    ("Random Forest", Pipeline([
        ("clf", RandomForestClassifier(random_state=42, n_jobs=-1))
    ]), {"clf__n_estimators": [100], "clf__max_depth": [None]}),

    ("Logistic Regression", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=500, random_state=42))
    ]), {"clf__C": [1]}),

    ("k-NN", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier())
    ]), {"clf__n_neighbors": [5, 7]}),

    ("Gaussian NB", Pipeline([
        ("clf", GaussianNB())
    ]), {})
]

# ──────────────────────────────────────────────────────────────
# 9. Train, tune and evaluate
# ──────────────────────────────────────────────────────────────
for name, pipe, param_grid in models:
    print(f"\n🔹 {name} Model 🔹")
    
    # Grid Search CV
    gs = GridSearchCV(pipe, param_grid, scoring="f1", cv=cv, n_jobs=-1)
    gs.fit(X, y, groups=groups)
    best_est = gs.best_estimator_

    # Evaluate with group-wise CV
    accs, precs, recs, f1s, aucs = [], [], [], [], []
    for tr, te in cv.split(X, y, groups):
        best_est.fit(X.iloc[tr], y[tr])
        preds = best_est.predict(X.iloc[te])
        probs = (best_est.predict_proba(X.iloc[te])[:, 1]
                 if hasattr(best_est, "predict_proba")
                 else best_est.decision_function(X.iloc[te]))

        accs.append(accuracy_score(y[te], preds))
        precs.append(precision_score(y[te], preds, zero_division=0))
        recs.append(recall_score(y[te], preds, zero_division=0))
        f1s.append(f1_score(y[te], preds, zero_division=0))
        aucs.append(roc_auc_score(y[te], probs))

    # Print metrics
    print(f"Best Params: {gs.best_params_}")
    print(f"Accuracy : {np.mean(accs):.3f} ± {np.std(accs):.3f}")
    print(f"Precision: {np.mean(precs):.3f} ± {np.std(precs):.3f}")
    print(f"Recall   : {np.mean(recs):.3f} ± {np.std(recs):.3f}")
    print(f"F1-score : {np.mean(f1s):.3f} ± {np.std(f1s):.3f}")
    print(f"ROC-AUC  : {np.mean(aucs):.3f} ± {np.std(aucs):.3f}")


✅ All selected features are present.
⚠️ 'Sex' column not found. Proceeding without it.

🔹 SVM (RBF) Model 🔹
Best Params: {'clf__C': 1, 'clf__gamma': 'scale'}
Accuracy : 0.852 ± 0.098
Precision: 0.862 ± 0.171
Recall   : 0.847 ± 0.051
F1-score : 0.845 ± 0.093
ROC-AUC  : 0.856 ± 0.110

🔹 Random Forest Model 🔹
Best Params: {'clf__max_depth': None, 'clf__n_estimators': 100}
Accuracy : 0.739 ± 0.106
Precision: 0.712 ± 0.141
Recall   : 0.770 ± 0.086
F1-score : 0.732 ± 0.100
ROC-AUC  : 0.788 ± 0.132

🔹 Logistic Regression Model 🔹
Best Params: {'clf__C': 1}
Accuracy : 0.678 ± 0.115
Precision: 0.676 ± 0.151
Recall   : 0.695 ± 0.103
F1-score : 0.669 ± 0.090
ROC-AUC  : 0.786 ± 0.118

🔹 k-NN Model 🔹
Best Params: {'clf__n_neighbors': 7}
Accuracy : 0.783 ± 0.091
Precision: 0.813 ± 0.159
Recall   : 0.737 ± 0.089
F1-score : 0.761 ± 0.083
ROC-AUC  : 0.835 ± 0.122

🔹 Gaussian NB Model 🔹
Best Params: {}
Accuracy : 0.748 ± 0.093
Precision: 0.758 ± 0.170
Recall   : 0.715 ± 0.066
F1-score : 0.727 ± 0.093
ROC